In [1]:
import rospy
from sensor_msgs.msg import PointCloud2
from visualization_msgs.msg import Marker, MarkerArray
from std_msgs.msg import ColorRGBA
from geometry_msgs.msg import Vector3
from pyquaternion import Quaternion
import sensor_msgs.point_cloud2 as pc2
from jsk_recognition_msgs.msg import BoundingBox, BoundingBoxArray
import logging
import sys
import time
sys.path.append("/home/johny/catkin_ws/src/second_ros/second.pytorch")

import numpy as np

import torch
from google.protobuf import text_format
from second.pytorch.train import build_network
from second.protos import pipeline_pb2
from second.utils import config_tool


class Second_ROS:
    def __init__(self):
        config_path, ckpt_path = self.init_ros()
        self.init_second(config_path, ckpt_path)

    def init_second(self, config_path, ckpt_path):
        """ Initialize second model """
        
        config = pipeline_pb2.TrainEvalPipelineConfig()
        with open(config_path, "r") as f:
            proto_str = f.read()
            text_format.Merge(proto_str, config)
        input_cfg = config.eval_input_reader
        model_cfg = config.model.second
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.net = build_network(model_cfg).to(self.device).eval()
        self.net.load_state_dict(torch.load(ckpt_path))
        target_assigner = self.net.target_assigner
        self.voxel_generator = self.net.voxel_generator

        grid_size = self.voxel_generator.grid_size
        feature_map_size = grid_size[:2] // config_tool.get_downsample_factor(model_cfg)
        feature_map_size = [*feature_map_size, 1][::-1]

        self.anchors = target_assigner.generate_anchors(feature_map_size)["anchors"]
        self.anchors = torch.tensor(self.anchors, dtype=torch.float32, device=self.device)
        self.anchors = self.anchors.view(1, -1, 7)
        print("[second_ros] Model Initialized")


    def init_ros(self):
        """ Initialize ros parameters """
            
        # Trained for pedestrians on custom dataset
        config_path = rospy.get_param("/config_path", "/home/johny/catkin_ws/src/second_ros/config/people.fhd.config")
        ckpt_path = rospy.get_param("/ckpt_path", "/home/johny/catkin_ws/src/second_ros/trained_custom_v2/voxelnet-55710.tckpt")
        
        return config_path, ckpt_path

    def inference(self, points):
        num_features = 4
        points = points.reshape([-1, num_features])
        dic = self.voxel_generator.generate(points)
        voxels, coords, num_points = dic['voxels'], dic['coordinates'], dic['num_points_per_voxel']
        coords = np.pad(coords, ((0, 0), (1, 0)), mode='constant', constant_values=0)
        voxels = torch.tensor(voxels, dtype=torch.float32, device=self.device)
        coords = torch.tensor(coords, dtype=torch.int32, device=self.device)
        num_points = torch.tensor(num_points, dtype=torch.int32, device=self.device)
        
        input_points = {
        "anchors": self.anchors,
        "voxels": voxels,
        "num_points": num_points,
        "coordinates": coords,
        }

        pred = self.net(input_points)[0]
        boxes_lidar = pred["box3d_lidar"].detach().cpu().numpy()
        scores = pred["scores"].detach().cpu().numpy()
        label = pred["label_preds"].detach().cpu().numpy()

        return boxes_lidar, scores, label

second = Second_ROS()

[  41 1280 1056]
[second_ros] Model Initialized


In [26]:
import os
from sensor_msgs.msg import PointCloud2, PointField
from std_msgs.msg import Header

rospy.init_node('second_ros_node')

LABEL_DIR = '/home/johny/CustomDatasetV2/training/label_2'
POINT_CLOUD_DIR = '/home/johny/CustomDatasetV2/training/velodyne'
CALIB_DIR = '/home/johny/CustomDatasetV2/training/calib'

# id for viewing
file_id = 0
iou_metric_2d = []
iou_metric_3d = []

for file_id in range(1):
    file_id = 90
    label_filename = os.path.join(LABEL_DIR, '{0:06d}.txt'.format(file_id))
    pc_filename = os.path.join(POINT_CLOUD_DIR, '{0:06d}.bin'.format(file_id))
    calib_filename = os.path.join(CALIB_DIR, '{0:06d}.txt'.format(file_id))

    import numpy as np

    point_cloud= np.fromfile(pc_filename, '<f4')  # little-endian float32
    point_cloud = np.reshape(point_cloud, (-1, 4))  # x, y, z, r

    # Create the PointCloud2 message
    pc_msg = PointCloud2()

    # Set the header
    pc_msg.header = Header()
    pc_msg.header.stamp = rospy.Time.now()
    pc_msg.header.frame_id = "rslidar"  # Replace "your_frame_id" with the desired frame ID

    # Set the fields
    pc_msg.fields.append(PointField(name="x", offset=0, datatype=PointField.FLOAT32, count=1))
    pc_msg.fields.append(PointField(name="y", offset=4, datatype=PointField.FLOAT32, count=1))
    pc_msg.fields.append(PointField(name="z", offset=8, datatype=PointField.FLOAT32, count=1))
    pc_msg.fields.append(PointField(name="rgb", offset=12, datatype=PointField.FLOAT32, count=1))

    # Set the point cloud data
    pc_msg.data = point_cloud.astype(np.float32).tobytes()

    # Set the width, height, and point_step
    pc_msg.width = len(point_cloud)
    pc_msg.height = 1
    pc_msg.point_step = 16  # Each point consists of 4 float32 values (x, y, z, rgb), each 4 bytes long

    # Set the is_dense flag (optional)
    pc_msg.is_dense = True  # Set to True if all the points in the point cloud are valid

    from jsk_recognition_msgs.msg import BoundingBox, BoundingBoxArray

    boxes_lidar, scores, label = second.inference(point_cloud)

    detected_bboxes = BoundingBoxArray()
    detected_bboxes.header = Header()
    detected_bboxes.header.frame_id = "rslidar"
    detected_bboxes.header.stamp    = rospy.Time.now()

    for box in boxes_lidar:
        bbox = BoundingBox()
        bbox.header = Header()
        bbox.header.frame_id = "rslidar"
        bbox.header.stamp    = rospy.Time.now()
        bbox.pose.position.x = float(box[0])
        bbox.pose.position.y = float(box[1])
        bbox.pose.position.z = float(box[2]) + float(box[5]) / 2 - 0.80
        bbox.dimensions.x    = float(box[3])
        bbox.dimensions.y    = float(box[4])
        bbox.dimensions.z    = float(box[5])
        detected_bboxes.boxes.append(bbox)

    import sys
    import os
    import numpy as np
    sys.path.append("/home/johny/catkin_ws/src/tools/src/")
    from label_viewer import camera_coordinate_to_point_cloud, load_kitti_calib

    label_bboxes = BoundingBoxArray()
    label_bboxes.header = Header()
    label_bboxes.header.frame_id = "rslidar"
    label_bboxes.header.stamp    = rospy.Time.now()
    
    calib = load_kitti_calib(calib_filename)

    with open(label_filename) as f_label:
        lines = f_label.readlines()
        for line in lines:
            line = line.strip('\n').split()
            box3d_center, box3d_corner = camera_coordinate_to_point_cloud(line[8:15], calib['Tr_velo_to_cam'])
            bbox = BoundingBox()
            bbox.header = Header()
            bbox.header.frame_id = "rslidar"
            bbox.header.stamp    = rospy.Time.now()
            bbox.pose.position.x = box3d_center[0][0]
            bbox.pose.position.y = box3d_center[0][1]
            bbox.pose.position.z = box3d_center[0][2] + float(np.linalg.norm(box3d_corner[0] - box3d_corner[4])) / 2
            bbox.dimensions.x    = np.linalg.norm(box3d_corner[0] - box3d_corner[1])
            bbox.dimensions.y    = np.linalg.norm(box3d_corner[1] - box3d_corner[2])
            bbox.dimensions.z    = np.linalg.norm(box3d_corner[0] - box3d_corner[4])
            label_bboxes.boxes.append(bbox)

    pub_point_cloud = rospy.Publisher("/synced_cloud", PointCloud2, queue_size=1)
    pub_detect_bbox = rospy.Publisher("/detections", BoundingBoxArray, queue_size=1)
    pub_label_bbox  = rospy.Publisher("/labels", BoundingBoxArray, queue_size=1)


    pub_point_cloud.publish(pc_msg)
    pub_detect_bbox.publish(detected_bboxes)
    pub_label_bbox.publish(label_bboxes)
    
#     import matplotlib.pyplot as plt
#     from scipy.optimize import linear_sum_assignment
#     cost_matrix = np.zeros((len(label_bboxes.boxes), len(detected_bboxes.boxes)))
#     for i, label_box in enumerate(label_bboxes.boxes):
#         for j, detected_box in enumerate(detected_bboxes.boxes):
#             # Calculate the Euclidean distance between the centers of the previous and current bounding boxes.
#             center_prev         = np.array([label_box.pose.position.x, label_box.pose.position.y, label_box.pose.position.z])
#             center_curr         = np.array([detected_box.pose.position.x, detected_box.pose.position.y, detected_box.pose.position.z])
#             cost_matrix[i, j]   = np.linalg.norm(center_prev - center_curr)
#             # Add the difference in size between the previous and current bounding boxes to the cost.
#             size_prev           = np.array([label_box.dimensions.x, label_box.dimensions.y, label_box.dimensions.z])
#             size_curr           = np.array([detected_box.dimensions.x, detected_box.dimensions.y, detected_box.dimensions.z])
#             size_diff           = np.linalg.norm(size_prev - size_curr)
#             cost_matrix[i, j]   += size_diff
#     row_ind, col_ind = linear_sum_assignment(cost_matrix)
#     matches = []
#     for i, j in zip(row_ind, col_ind):
#         if cost_matrix[i, j] > 1:
#             print("Bounding box matched above threshold!")
#             continue
#         matches.append((i, j))

#     for match in matches:
#         # Calculate the coordinates of the intersection cuboid in 3D (x, y, z)
#         x1 = max(label_bboxes.boxes[match[0]].pose.position.x - label_bboxes.boxes[match[0]].dimensions.x / 2, 
#                 detected_bboxes.boxes[match[1]].pose.position.x - detected_bboxes.boxes[match[1]].dimensions.x / 2)
#         y1 = max(label_bboxes.boxes[match[0]].pose.position.y - label_bboxes.boxes[match[0]].dimensions.y / 2, 
#                 detected_bboxes.boxes[match[1]].pose.position.y - detected_bboxes.boxes[match[1]].dimensions.y / 2)
#         z1 = max(label_bboxes.boxes[match[0]].pose.position.z - label_bboxes.boxes[match[0]].dimensions.z / 2, 
#                 detected_bboxes.boxes[match[1]].pose.position.z - detected_bboxes.boxes[match[1]].dimensions.z / 2)
#         x2 = min(label_bboxes.boxes[match[0]].pose.position.x + label_bboxes.boxes[match[0]].dimensions.x / 2, 
#                 detected_bboxes.boxes[match[1]].pose.position.x + detected_bboxes.boxes[match[1]].dimensions.x / 2)
#         y2 = min(label_bboxes.boxes[match[0]].pose.position.y + label_bboxes.boxes[match[0]].dimensions.y / 2, 
#                 detected_bboxes.boxes[match[1]].pose.position.y + detected_bboxes.boxes[match[1]].dimensions.y / 2)
#         z2 = min(label_bboxes.boxes[match[0]].pose.position.z + label_bboxes.boxes[match[0]].dimensions.z / 2, 
#                 detected_bboxes.boxes[match[1]].pose.position.z + detected_bboxes.boxes[match[1]].dimensions.z / 2)

#         # If the intersection is non-existent (negative volume), return IoU of 0
#         if x2 <= x1 or y2 <= y1 or z2 <= z1:
#             iou = 0.0
#             print("IoU -> 0")

#         # Calculate the volumes of the bounding cuboids
#         volume_bbox1 = (label_bboxes.boxes[match[0]].dimensions.x * label_bboxes.boxes[match[0]].dimensions.y *
#                         label_bboxes.boxes[match[0]].dimensions.z)
#         volume_bbox2 = (detected_bboxes.boxes[match[1]].dimensions.x * detected_bboxes.boxes[match[1]].dimensions.y *
#                         detected_bboxes.boxes[match[1]].dimensions.z)

#         # Calculate the volume of intersection
#         intersection_volume = (x2 - x1) * (y2 - y1) * (z2 - z1)

#         # Calculate the volume of union
#         union_volume = volume_bbox1 + volume_bbox2 - intersection_volume

#         # Calculate the IoU
#         iou = intersection_volume / union_volume
#         if iou > 0.5:
#             iou_metric_3d.append(iou)


# iou_values = np.array(iou_metric_3d)


# # Function to plot the graph
# plt.plot(iou_values)
# plt.ylim(0, 1)
# plt.xlabel('Iteration')
# plt.ylabel('IoU')
# plt.title('IoU per Iteration')
# plt.gcf().set_figwidth(7)  # Adjust the width and height as needed
# plt.subplots_adjust(left=0.08, bottom=0.12, right=0.98, top=0.92)  # Adjust the margins as needed
# plt.savefig('/home/johny/catkin_ws/src/tools/matplots/run/iou_3d.jpeg', dpi=600)
# plt.show()

# average_iou = np.mean(iou_values)
# std_dev = np.std(iou_values)
# print("Average IoU: ", average_iou)
# print("Std Dev: ", std_dev)

# # Calculate estimation error (in IoU)
# estimation_errors = 1 - iou_values

# # Function to plot the graph
# plt.plot(estimation_errors)
# plt.ylim(0, 1)
# plt.xlabel('Iteration')
# plt.ylabel('Estimation Error')
# plt.title('Estimation Error per Iteration')
# plt.gcf().set_figwidth(7)  # Adjust the width and height as needed
# plt.subplots_adjust(left=0.08, bottom=0.12, right=0.98, top=0.92)  # Adjust the margins as needed
# plt.savefig('/home/johny/catkin_ws/src/tools/matplots/run/iou_error_3d.jpeg', dpi=600)
# plt.show()

# # Calculate average estimation error
# average_estimation_error = np.mean(estimation_errors)
# std_dev = np.std(estimation_errors)
# print("Average Error: ", average_estimation_error)
# print("Std Dev: ", std_dev)
